In [2]:
from torcheeg import transforms
from torcheeg.datasets import NumpyDataset
import numpy as np

# Mock 100 EEG samples. Each EEG signal contains a signal of length 1 s at a sampling rate of 128 sampled by 32 electrodes.
X = np.random.randn(100, 32, 128)
X.shape

(100, 32, 128)

In [6]:
# Mock 100 labels, denoting valence and arousal of subjects during EEG recording.
y = {
    'valence': np.random.randint(10, size=100),
    'arousal': np.random.randint(10, size=100)
}
y['arousal']

array([7, 2, 9, 7, 7, 0, 9, 1, 9, 9, 8, 2, 4, 6, 1, 9, 0, 2, 9, 7, 1, 8,
       6, 6, 2, 5, 9, 9, 7, 7, 6, 1, 9, 9, 7, 9, 3, 7, 2, 0, 6, 5, 6, 3,
       6, 3, 5, 8, 9, 9, 7, 9, 4, 3, 4, 0, 8, 4, 2, 2, 4, 5, 9, 8, 3, 8,
       1, 7, 1, 0, 7, 2, 8, 2, 9, 3, 4, 8, 1, 2, 3, 5, 5, 5, 1, 8, 1, 3,
       4, 6, 5, 0, 5, 2, 1, 0, 3, 6, 9, 9])

In [7]:
dataset = NumpyDataset(X=X,
                       y=y,
                       offline_transform=transforms.Compose(
                           [transforms.BandDifferentialEntropy()]),
                       online_transform=transforms.ToTensor(),
                       label_transform=transforms.Compose([
                           transforms.Select('valence'),
                           transforms.Binary(5.0),
                       ]),
                       num_worker=2,
                       num_samples_per_worker=50)
print(dataset[0])
# EEG signal (torch.Tensor[32, 4]),
# coresponding baseline signal (torch.Tensor[32, 4]),
# label (int)

[NUMPY]: 100%|██████████| 2/2 [00:06<00:00,  3.14s/it]

Please wait for the writing process to complete...
(tensor([[0.9977, 1.0947, 1.5477, 1.6099],
        [0.7360, 1.0672, 1.5667, 1.5554],
        [1.1181, 1.3062, 1.5068, 1.6172],
        [0.9989, 0.9548, 1.4552, 1.6272],
        [0.9546, 1.3517, 1.6392, 1.5390],
        [0.9426, 1.3396, 1.5470, 1.6258],
        [0.0276, 1.0254, 1.5488, 1.5520],
        [0.8091, 1.0522, 1.4938, 1.4019],
        [0.9161, 1.2366, 1.4505, 1.5125],
        [0.6656, 0.5364, 1.5305, 1.5900],
        [0.6386, 1.2520, 1.4476, 1.5505],
        [0.8092, 0.9804, 1.4645, 1.5617],
        [0.7415, 1.1631, 1.5099, 1.4905],
        [0.8775, 1.1523, 1.6474, 1.5894],
        [1.0858, 1.3291, 1.6614, 1.6697],
        [0.6647, 0.8383, 1.5668, 1.6706],
        [0.8262, 1.0731, 1.6153, 1.6386],
        [1.1335, 1.2174, 1.5927, 1.5181],
        [0.9449, 1.1123, 1.5764, 1.4768],
        [0.9190, 1.3252, 1.5612, 1.5460],
        [1.1367, 1.2181, 1.5484, 1.5296],
        [1.1622, 1.1012, 1.4263, 1.5645],
        [1.0555, 1.3531,